In [46]:
import pandas as pd
import requests
import json
import numpy as np

In [ ]:
def getAllStreamStats(i):
    ## Define gage location
#     y_loc = dat["dec_lat_va"][i]
#     x_loc = dat["dec_long_va"][i]
    
    y_loc = np.round(dat["dec_lat_va"][i],2)
    x_loc = np.round(dat["dec_long_va"][i],2)
    state = 'OR'
    gage = dat["gage"][i]
    
    
    ## Define Streamstats API URLS
    # Service URLS
    StreamStatsServiceURLS = {
        'watershed': 'https://streamstats.usgs.gov/streamstatsservices/watershed.geojson',
        'basinCharacteristics': 'https://streamstats.usgs.gov/streamstatsservices/parameters.json',
        'gage': 'https://streamstats.usgs.gov/gagestatsservices/stations'
        }
    NSSServiceURlS = {
        'regressionRegions': 'https://streamstats.usgs.gov/nssservices/regressionregions/bylocation',
        'scenarios': 'https://streamstats.usgs.gov/nssservices/scenarios',
        'computeFlowStats': 'https://streamstats.usgs.gov/nssservices/scenarios/Estimate'
    }

    # Declare cookies for use of consistent StreamStats servers
    global cookies 
    
    # Step 1 - Get the watershed
    print('Step 1 - Get the watershed')
    watershedURLParams = {
        'rcode': state, 
        'xlocation': x_loc, 
        'ylocation': y_loc,
        'crs': 4326,
    #     'crs': 4269,
    }
    delineatedBasin = None
    while delineatedBasin is None: # Sometimes the watershed endpoint returns with no geometry - this loop is written to keep trying until a geometry is returned
        print('Delineating basin...')
        basinDelineationResponse = requests.get(url = StreamStatsServiceURLS['watershed'], params = watershedURLParams)
        if basinDelineationResponse.status_code == 200:
            cookies = basinDelineationResponse.cookies
            try:
                delineatedBasin = json.loads(basinDelineationResponse.content.decode('utf-8'))["featurecollection"][1]["feature"]["features"][0]["geometry"] # this will be used in step 2
                workspaceID = json.loads(basinDelineationResponse.content.decode('utf-8'))["workspaceID"] # this will be used in step 4
#             except:
#                 print("No geometry returned. Retrying...")
#                 pass
        else:
            print("Error. Retrying...")
            pass

    # Step 2 - Get the Regression Regions associated with the watershed
    print('Step 2 - Get the Regression Regions associated with the watershed')
    regressionRegionPOSTBody = delineatedBasin # from step 1

    regressionRegions = None
    while regressionRegions == None:
        regressionRegionsResponse = requests.post(url = NSSServiceURlS['regressionRegions'], json=regressionRegionPOSTBody)
        if regressionRegionsResponse.status_code == 200:
            regressionRegions = json.loads(regressionRegionsResponse.content.decode('utf-8'))
            regressionRegionCodes = [ sub['code'] for sub in regressionRegions ] # this will be used in step 3
        else:
            print("Error.")
            pass

    # Step 3 - Get the Scenarios associated with the Regression Regions
    print('Step 3 - Get the Scenarios associated with the Regression Regions')

    scenarioURLParams = {
        'regions': state, 
        'statisticgroups': 4, # this can be updated depending on which flow statistics you want to calculate (2 is associated with Peak Flow Statistics) - you can find all statistic groups available for your region at https://streamstats.usgs.gov/docs/nssservices/#/StatisticGroups/GET/StatisticGroups
        'regressionregions': (','.join(regressionRegionCodes)) # from step 2, needs to be comma separated list
    }
    scenarioResponse = requests.get(url = NSSServiceURlS['scenarios'], params=scenarioURLParams)

    parameters = None
    while parameters == None:
        if scenarioResponse.status_code == 200:
            scenarios = json.loads(scenarioResponse.content.decode('utf-8'))[0] # this will be used in step 5
            parameters = json.loads(scenarioResponse.content.decode('utf-8'))[0]["regressionRegions"][0]["parameters"]
            parameterCodes = [ sub['code'] for sub in parameters ] # this will be used in step 4
        else:
            print("Error.")
            pass

    # Step 4 - Compute the basin characteristics
    print('Step 4 - Compute the basin characteristics')
    basinCharacteristicsURLParams = {
        'rcode': state, 
        'workspaceID': workspaceID, # from step 1
        'includeparameters': (','.join(parameterCodes)) # from step 3
    }

    basinCharacteristics = None
    while basinCharacteristics is None:
        basinCharacteristicsResponse = requests.get(url = StreamStatsServiceURLS['basinCharacteristics'], params=basinCharacteristicsURLParams, cookies=cookies)
        if basinCharacteristicsResponse.status_code == 200:
            basinCharacteristics = json.loads(basinCharacteristicsResponse.content.decode('utf-8')) # this will be used in step 5
        else:
            print("Error.")
            pass

    # Step 5 - Compute Flow Statistics
    print('Step 5 - Compute Flow Statistics')
    flowStatsURLParams = {
        'regions': state
    }
    flowStatsPOSTBody = [] 
    for counter, x in enumerate(scenarios['regressionRegions'][0]['parameters']): # from step 3 & step 4
        for p in basinCharacteristics['parameters']:
            if x['code'].lower() == p['code'].lower():
                scenarios['regressionRegions'][0]['parameters'][counter]['value'] = p['value']
    flowStatsPOSTBody.append(scenarios)

    flowStats = None
    while flowStats == None:
        flowStatsResponse = requests.post(url = NSSServiceURlS['computeFlowStats'], params = flowStatsURLParams, json=flowStatsPOSTBody)
        if flowStatsResponse.status_code == 200:
            flowStats = json.loads(flowStatsResponse.content.decode('utf-8'))
        else:
            print("Error.")
            pass

    out = pd.json_normalize(flowStats,
                             record_path=['regressionRegions',['results']],
                            )[["code","value"]]
    out["gage"] = gage
    return out

In [53]:
dat[dat['site_no']==12115700]

,site_no,dec_lat_va,dec_long_va,gage,numZero,numOne,startDate,endDate,comid,numObs,propZero,propOne,geometry
63,12115700,47.366216,-121.692882,12115700,1128,1888,1983-03-02,2020-12-31,24538136.0,13819,0.081627,0.136623,POINT (-121.6928821 47.36621625)


In [57]:
i=63
getAllStreamStats(i)

Step 1 - Get the watershed
Delineating basin...
No geometry returned. Retrying...
Delineating basin...


KeyboardInterrupt: 

In [26]:
## Define gage location
y_loc = dat["dec_lat_va"][i]
x_loc = dat["dec_long_va"][i]
state = 'OR'
gage = dat["gage"][i]


## Define Streamstats API URLS
# Service URLS
StreamStatsServiceURLS = {
    'watershed': 'https://streamstats.usgs.gov/streamstatsservices/watershed.geojson',
    'basinCharacteristics': 'https://streamstats.usgs.gov/streamstatsservices/parameters.json',
    'gage': 'https://streamstats.usgs.gov/gagestatsservices/stations'
    }
NSSServiceURlS = {
    'regressionRegions': 'https://streamstats.usgs.gov/nssservices/regressionregions/bylocation',
    'scenarios': 'https://streamstats.usgs.gov/nssservices/scenarios',
    'computeFlowStats': 'https://streamstats.usgs.gov/nssservices/scenarios/Estimate'
}

# Declare cookies for use of consistent StreamStats servers
global cookies 

# Step 1 - Get the watershed
print('Step 1 - Get the watershed')
watershedURLParams = {
    'rcode': state, 
    'xlocation': x_loc, 
    'ylocation': y_loc,
    'crs': 4326,
#     'crs': 4269,
}
delineatedBasin = None
while delineatedBasin is None: # Sometimes the watershed endpoint returns with no geometry - this loop is written to keep trying until a geometry is returned
    print('Delineating basin...')
    basinDelineationResponse = requests.get(url = StreamStatsServiceURLS['watershed'], params = watershedURLParams)
    if basinDelineationResponse.status_code == 200:
        cookies = basinDelineationResponse.cookies
        try:
            delineatedBasin = json.loads(basinDelineationResponse.content.decode('utf-8'))["featurecollection"][1]["feature"]["features"][0]["geometry"] # this will be used in step 2
            workspaceID = json.loads(basinDelineationResponse.content.decode('utf-8'))["workspaceID"] # this will be used in step 4
        except:
            print("No geometry returned. Retrying...")
    else:
        print("Error. Retrying...")

# Step 2 - Get the Regression Regions associated with the watershed
print('Step 2 - Get the Regression Regions associated with the watershed')
regressionRegionPOSTBody = delineatedBasin # from step 1

regressionRegions = None
while regressionRegions == None:
    regressionRegionsResponse = requests.post(url = NSSServiceURlS['regressionRegions'], json=regressionRegionPOSTBody)
    if regressionRegionsResponse.status_code == 200:
        regressionRegions = json.loads(regressionRegionsResponse.content.decode('utf-8'))
        regressionRegionCodes = [ sub['code'] for sub in regressionRegions ] # this will be used in step 3
    else:
        print("Error.")
        pass

# Step 3 - Get the Scenarios associated with the Regression Regions
print('Step 3 - Get the Scenarios associated with the Regression Regions')

scenarioURLParams = {
    'regions': state, 
    'statisticgroups': 4, # this can be updated depending on which flow statistics you want to calculate (2 is associated with Peak Flow Statistics) - you can find all statistic groups available for your region at https://streamstats.usgs.gov/docs/nssservices/#/StatisticGroups/GET/StatisticGroups
    'regressionregions': (','.join(regressionRegionCodes)) # from step 2, needs to be comma separated list
}
scenarioResponse = requests.get(url = NSSServiceURlS['scenarios'], params=scenarioURLParams)

parameters = None
while parameters == None:
    if scenarioResponse.status_code == 200:
        scenarios = json.loads(scenarioResponse.content.decode('utf-8'))[0] # this will be used in step 5
        parameters = json.loads(scenarioResponse.content.decode('utf-8'))[0]["regressionRegions"][0]["parameters"]
        parameterCodes = [ sub['code'] for sub in parameters ] # this will be used in step 4
    else:
        print("Error.")
        pass

# Step 4 - Compute the basin characteristics
print('Step 4 - Compute the basin characteristics')
basinCharacteristicsURLParams = {
    'rcode': state, 
    'workspaceID': workspaceID, # from step 1
    'includeparameters': (','.join(parameterCodes)) # from step 3
}

basinCharacteristics = None
while basinCharacteristics is None:
    basinCharacteristicsResponse = requests.get(url = StreamStatsServiceURLS['basinCharacteristics'], params=basinCharacteristicsURLParams, cookies=cookies)
    if basinCharacteristicsResponse.status_code == 200:
        basinCharacteristics = json.loads(basinCharacteristicsResponse.content.decode('utf-8')) # this will be used in step 5
    else:
        print("Error.")
        pass

# Step 5 - Compute Flow Statistics
print('Step 5 - Compute Flow Statistics')
flowStatsURLParams = {
    'regions': state
}
flowStatsPOSTBody = [] 
for counter, x in enumerate(scenarios['regressionRegions'][0]['parameters']): # from step 3 & step 4
    for p in basinCharacteristics['parameters']:
        if x['code'].lower() == p['code'].lower():
            scenarios['regressionRegions'][0]['parameters'][counter]['value'] = p['value']
flowStatsPOSTBody.append(scenarios)

flowStats = None
while flowStats == None:
    flowStatsResponse = requests.post(url = NSSServiceURlS['computeFlowStats'], params = flowStatsURLParams, json=flowStatsPOSTBody)
    if flowStatsResponse.status_code == 200:
        flowStats = json.loads(flowStatsResponse.content.decode('utf-8'))
    else:
        print("Error.")
        pass

out = pd.json_normalize(flowStats,
                         record_path=['regressionRegions',['results']],
                        )[["code","value"]]
out["gage"] = gage

Step 1 - Get the watershed
Delineating basin...
Step 2 - Get the Regression Regions associated with the watershed
Step 3 - Get the Scenarios associated with the Regression Regions
Step 4 - Compute the basin characteristics
Step 5 - Compute Flow Statistics


In [36]:
scenarioURLParams

{'regions': 'OR',
 'statisticgroups': 4,
 'regressionregions': 'GC1290,GC1410,GC729,GC1340,GC1825,GC1330,GC1855,GC1400,GC1390,GC1320,GC1370,GC1300,GC1380,GC1360,GC1350,GC1310,GC730,GC731,GC1826,GC1910,GC1828'}

In [34]:
DRNAREA = 0.66
MINTEMP = 41
1.10745*(10**11.5732)*(DRNAREA**0.9144)*((MINTEMP-20)**(-9.4485))-1

-0.9088983808327197

In [3]:
# dat = pd.read_csv("../data/pnwNP_StatsLocations.csv")

# from tqdm import tqdm
# # for i in tqdm(range(len(pnwNP)+1)):
# for i in tqdm(range(5)):
#     try:
#         out = getAllStreamStats(i)
        
#     except:
#         pass

  0%|                                                     | 0/5 [00:00<?, ?it/s]

Step 1 - Get the watershed
Delineating basin...
No geometry returned. Retrying...
Delineating basin...
Step 2 - Get the Regression Regions associated with the watershed
Step 3 - Get the Scenarios associated with the Regression Regions
Step 4 - Compute the basin characteristics
Step 5 - Compute Flow Statistics


 20%|████████▊                                   | 1/5 [02:52<11:29, 172.32s/it]

Step 1 - Get the watershed
Delineating basin...
Step 2 - Get the Regression Regions associated with the watershed
Step 3 - Get the Scenarios associated with the Regression Regions
Step 4 - Compute the basin characteristics
Step 5 - Compute Flow Statistics


 40%|█████████████████▌                          | 2/5 [04:23<06:13, 124.64s/it]

...

 60%|██████████████████████████▍                 | 3/5 [05:50<03:34, 107.37s/it]

Step 1 - Get the watershed
Delineating basin...
Step 2 - Get the Regression Regions associated with the watershed
Step 3 - Get the Scenarios associated with the Regression Regions
Step 4 - Compute the basin characteristics
Step 5 - Compute Flow Statistics


 80%|████████████████████████████████████         | 4/5 [07:08<01:35, 95.89s/it]

Step 1 - Get the watershed
Delineating basin...
Step 2 - Get the Regression Regions associated with the watershed
Step 3 - Get the Scenarios associated with the Regression Regions
Step 4 - Compute the basin characteristics
Step 5 - Compute Flow Statistics


100%|████████████████████████████████████████████| 5/5 [08:43<00:00, 104.69s/it]
